# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [27]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable
from IPython.display import display, HTML

#### Creating list of filepaths to process original event csv data files

In [28]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-08-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [40]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 


    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

print(len(full_data_rows_list))


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f,dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        #print(row)
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [41]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [31]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
data=pd.read_csv('event_datafile_new.csv')
from cassandra.cluster import Cluster
cluster = Cluster()
# To establish connection and begin executing queries, need a session
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
        print(e)



#### Create Keyspace

In [32]:

try:
    session.execute(
    "CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")    
except Exception as e:
        print(e)

#### Set Keyspace

In [33]:
try:
    session.set_keyspace('udacity')
except Exception as e:
        print(e)
   

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### **FUNCTIONS**
##### There are three functions for creating tables,inserting data and for select queries to verify if the data is properly inserted into tables.


In [34]:

def create_table(table_name):
    ''' This will create the table with specified table name 
         and adds the columns which are required for that table from the
         csv file.'''
    
    query = "CREATE TABLE IF NOT EXISTS "+table_name
    if(table_name=="session_history"):
        query = query + "(sessionId int, itemInSession int, artist text, song text,length double, PRIMARY KEY (sessionId,itemInSession))"

    elif(table_name=="user_history"):
        query= query + "( userId int,sessionId int,itemInSession int,firstName text,lastName text,artist text,song text,PRIMARY KEY ((userId,sessionId),itemInSession)) ;"
    
    elif(table_name=="song_history"):
        query= query + "(song text,userId int,firstName text,lastName text,PRIMARY KEY (song,userId));"

    try:
        session.execute(query)
    except Exception as e:
        print(e)
        
def insert_table(table_name):
    ''' Depending upon which table name is passed this will create insert 
        query,then pass the corresponding columns to the insert statement to
        insert the records into the table.'''
    
    ## TO-DO: Assign the INSERT statements into the `query` variable depending on table name

    query=""
    file="event_datafile_new.csv"
    if(table_name=="session_history"):
        query = "INSERT INTO session_history (sessionId,itemInSession,artist,song,length)"
        query = query + " VALUES (%s, %s, %s, %s,%s);"
    elif (table_name=="user_history"):
        query = "INSERT INTO user_history ( userId,itemInSession,sessionId,firstName,lastName,artist,song)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s);"
    elif(table_name=="song_history"):
        query="INSERT INTO song_history (song,userId,firstName,lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
    
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
    
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
            if(table_name=="session_history"):
                session.execute(query, (int(line[8]),int(line[3]),str(line[0]),str(line[9]),float(line[5])))
            elif(table_name=="user_history"):
                session.execute(query, (int(line[10]),int(line[3]),int(line[8]),line[1],line[4],str(line[0]),line[9]))
            elif(table_name=="song_history"):
                session.execute(query, (str(line[9]),int(line[10]),line[1],line[4],))
                        
def verify_select(table_name):
    ''' This function is used to select all rows from the table
    to verify if the data is properly inserted.'''
    query="select * from "+ table_name 
    df=pd.DataFrame(list(session.execute(query)))
    if(df.empty):
        print("No records present in the table "+ table_name)
    else:
        df2=df.head()
        display(HTML(df2.to_html()))


### SESSION_HISTORY TABLE
##### **Query 1**:Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

##### **Solution** :For first query session_history table is created which has columns sessionId,itemInsession,artist name,length,song title. *sessionId* is used as Partition Key since we are looking for specific records which has sessionId 182. *itemInSession* is used as Clustering key.

In [35]:
# Create table session_history.
create_table("session_history")

#insert data into session_history table.
insert_table("session_history")

print("===========================================================")
print("Verify if records are inserted by printing few records.")
print("===========================================================")

verify_select("session_history")

query = "select artist,song,length from session_history WHERE sessionId = 338 and itemInSession = 4"

#Use prettytables  to display the results.
t = PrettyTable(['Artist','Song','length'])

try:
    rows=session.execute(query)
    for row in rows:
        t.add_row([row.artist,row.song,row.length])


except Exception as e:
    print(e)
print("===========================================================")
print("Results of first query: ")
print("===========================================================")

print(t)




Verify if records are inserted by printing few records.


,sessionid,iteminsession,artist,length,song
0,23,0,Regina Spektor,191.08526,The Calculation (Album Version)
1,23,1,Octopus Project,250.95791,All Of The Champs That Ever Lived
2,23,2,Tegan And Sara,180.06159,So Jealous
3,23,3,Dragonette,153.39057,Okay Dolores
4,23,4,Lil Wayne / Eminem,229.58975,Drop The World


Results of first query: 
+-----------+---------------------------------+----------+
|   Artist  |               Song              |  length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


### USER_HISTORY TABLE

##### Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### **Solution**:For second query user_history table is created which has columns firstName,lastName,artist,song,sessionId,itemInsession,artist name. *userId* and *sessionId* is used as Composite Key as and *itemInSession* is used as Clustering key as records needs to be sorted according to it.


In [36]:
# Create table user_history
create_table("user_history")

#insert data into the table
insert_table("user_history")

print("===========================================================")
print("Verify if records are inserted by printing few records.")
print("===========================================================")
verify_select("user_history")


query = "select artist,song,firstName,lastName from user_history WHERE userId = 10 and sessionId =182"

#Use prettytables  to display the results.

t = PrettyTable(['Artist','Song','FirstName','LastName'])

try:
    rows=session.execute(query)
    for row in rows:
        t.add_row([row.artist,row.song,row.firstname,row.lastname])

except Exception as e:
    print(e)
print("===========================================================")
print("Results of second query: ")
print("===========================================================")
print(t)

Verify if records are inserted by printing few records.


,userid,sessionid,iteminsession,artist,firstname,lastname,song
0,58,768,0,System of a Down,Emily,Benson,Sad Statue
1,58,768,1,Ghostland Observatory,Emily,Benson,Stranger Lover
2,58,768,2,Evergreen Terrace,Emily,Benson,Zero
3,85,776,2,Deftones,Kinsley,Young,Head Up (LP Version)
4,85,776,3,The Notorious B.I.G.,Kinsley,Young,Playa Hater (Amended Version)


Results of second query: 
+-------------------+------------------------------------------------------+-----------+----------+
|       Artist      |                         Song                         | FirstName | LastName |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+



### SONG_HISTORY TABLE

##### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


##### **Solution**: For third query song_history table is created which has columns  userId,firstName,lastName,song title. For this query we have to use *song* as a primary key as query requires us to have the unique records for that.userId is used as Clustering key so that records are clustered using userId.


In [37]:
create_table("song_history")
insert_table("song_history")

#SELECT to verify that the data have been inserted into each table
print("===========================================================")
print("Verify if records are inserted by printing few records.")
print("===========================================================")

verify_select("song_history")
query = "select firstName,lastName from song_history WHERE song='All Hands Against His Own'"
df=pd.DataFrame()
t = PrettyTable(['FirstName','LastName'])

try:
    rows=session.execute(query)
    for row in rows:
        t.add_row([row.firstname,row.lastname])


except Exception as e:
    print(e)
print("===========================================================")
print("Results of third query: ")
print("===========================================================")

print(t)
                    

Verify if records are inserted by printing few records.


,song,userid,firstname,lastname
0,Wonder What's Next,49,Chloe,Cuevas
1,In The Dragon's Den,49,Chloe,Cuevas
2,Too Tough (1994 Digital Remaster),44,Aleena,Kirby
3,Rio De Janeiro Blue (Album Version),49,Chloe,Cuevas
4,My Place,15,Lily,Koch


Results of third query: 
+------------+----------+
| FirstName  | LastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the tables before closing out the sessions

In [38]:
## Drop the table before closing out the sessions

query = "drop table IF EXISTS session_history"
query1 = "drop table IF EXISTS user_history"
query2 = "drop table IF EXISTS song_history"

try:
    rows = session.execute(query)
    rows1 = session.execute(query1)
    rows2 = session.execute(query2)


except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [39]:
session.shutdown()
cluster.shutdown()